# Using databases with python- week 4

From [coursera](https://www.coursera.org/learn/python-databases).

July 2022.


In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

<div id="toc"></div>

# Many-to-many relationships

Everything we have done so far is "one-to-many", such as, an album has many tracks. We have a foreign key on the many side (tracks) pointing to the primary key in the one side (album).

Sometimes we have a relationship between data that is many-to-many. For example, there can be many artists on an album. 

Look at another example, books and authors. In this diagram, the crows feet shows that there is a many-to-many relationship. An author can write many books, and a book can contain many authors.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/c4/CPT-Databases-ManytoMany.svg/1024px-CPT-Databases-ManytoMany.svg.png?modified=1" />

In this case, you can't put a single foreign key on either side pointing to the other. So you need to create a table in the middle that models the relationship between them. This turns it into two one-to-many relationships. 

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/02/Databases-ManyToManyWJunction.jpg/1024px-Databases-ManyToManyWJunction.jpg?modified=1" />

# Set up a database with a many-to-many relationship

Let's model this table in the middle for another example- students taking classes. 


<img src="img/week4_01.jpg?modified=1" />

The table in the middle is called the membership table and has two foreign keys in it. Each row has two foreign keys, so each connection between course and user has one row in the membership table.

We don't put a primary key in the membership table because we can instead make a key that is a combination of the two foreign keys. You could however add another entry called "role" that stores whether the user is the instructor or student. 

Let's code up an example in a database. I'm going to use `DBeaver` for this because I want to be able to visualize it easily.

`CREATE TABLE Users (
    id     INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name   TEXT,
    email  TEXT
);
CREATE TABLE Course (
    id     INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    title  TEXT
);
CREATE TABLE Members (
    user_id    INTEGER,
    course_id  INTEGER,
    role  INTEGER,
    PRIMARY KEY (user_id, course_id)
);`

Note that SQL treats `role` as something special, not just your normal column name. 

Create some entries in the `Users` and `Course` tables.

`INSERT INTO Users (name, email) VALUES ('Patricia','pfschus@umich.edu');  
INSERT INTO Users (name, email) VALUES ('Charlie','caschulx@umich.edu');  
INSERT INTO Users (name, email) VALUES ('Cassie','ruchcas@umich.edu');`

`INSERT INTO Course (title) VALUES ('Eng 101');  
INSERT INTO Course (title) VALUES ('NERS 250');  
INSERT INTO Course (title) VALUES ('NERS 499');  `



How do we populate the `Member` table? Connecting each user to a course and assigning a role. 

In this case, we are going to use `0 = student` and `1 = instructor` for the role. In this case, I'm setting up the following:

* All three of us were students in Eng 101  
* Cassie and Patricia were students in NERS 250  
* Patricia was an instructor in NERS 499  

`INSERT INTO Members (user_id, course_id, role) VALUES (1,1,0);
INSERT INTO Members (user_id, course_id, role) VALUES (1,2,0);
INSERT INTO Members (user_id, course_id, role) VALUES (1,3,1);
INSERT INTO Members (user_id, course_id, role) VALUES (2,1,0);
INSERT INTO Members (user_id, course_id, role) VALUES (3,1,0);
INSERT INTO Members (user_id, course_id, role) VALUES (3,2,0);`

Here is what all of the tables look like individually. 


<img src="img/week4_02.jpg?modified=1" />

# Get data out of that database

We will use a JOIN statement to put these three tables together into something useful for us. 

Follow the same logic as in week 3:  
* What we want to see: `SELECT Users.name, Users.email, Members.role, Course.title`
* The tables that hold the data: `FROM Users JOIN Members JOIN Course`
* How the tables are linked: `ON Members.user_id = Users.id AND Members.course_id = Course.id`
* *BONUS* Order the entries: `ORDER BY Course.title, Member.role DESC, Users.name`
    * This sets the precedent about what's the most important information

<img src="img/week4_03.jpg?modified=1" />

# Why go to all these lengths

Why do we spend so much time to carefully set up all of these tables? It's all about speed. 

At a certain point, you cross over a point where it starts to get slow. The only way to get it to go fast is by rearchitecting it. 

In databases, if you set it up properly to begin with, it should always be fast. 